In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/google-gemma-3n-hackathon/README
/kaggle/input/google-gemma-3n-hackathon/archive/README
/kaggle/input/fieldscribe-assets/tomato_leaf.jpg
/kaggle/input/fieldscribe-assets/farmer_note.wav
/kaggle/input/paligemma-2/transformers/paligemma2-3b-mix-448/1/model.safetensors.index.json
/kaggle/input/paligemma-2/transformers/paligemma2-3b-mix-448/1/config.json
/kaggle/input/paligemma-2/transformers/paligemma2-3b-mix-448/1/model-00001-of-00002.safetensors
/kaggle/input/paligemma-2/transformers/paligemma2-3b-mix-448/1/model-00002-of-00002.safetensors
/kaggle/input/paligemma-2/transformers/paligemma2-3b-mix-448/1/preprocessor_config.json
/kaggle/input/paligemma-2/transformers/paligemma2-3b-mix-448/1/tokenizer.json
/kaggle/input/paligemma-2/transformers/paligemma2-3b-mix-448/1/tokenizer_config.json
/kaggle/input/paligemma-2/transformers/paligemma2-3b-mix-448/1/special_tokens_map.json
/kaggle/input/paligemma-2/transformers/paligemma2-3b-mix-448/1/generation_config.json


# Install necessary libraries

In [2]:
!pip install timm --upgrade
!pip install accelerate
!pip install git+https://github.com/huggingface/transformers.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━

#  Setup and Imports


In [3]:
# =================================================================
# CELL 1: SETUP AND IMPORTS (WITH SPEECH-TO-TEXT)
# =================================================================

# Install the powerful whisper library for speech recognition
!pip install -q openai-whisper

import os
os.environ['TORCH_DYNAMO_DISABLE'] = '1'

import torch
import ipywidgets as widgets
import io
import whisper # <-- Import whisper
import tempfile # <-- For handling the audio file
from PIL import Image
from IPython.display import display, Markdown
from transformers import PaliGemmaForConditionalGeneration, PaliGemmaProcessor

print("✅ Libraries (including Whisper for audio) imported. Dynamo disabled.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 10.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


2025-08-01 08:15:50.742339: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754036151.023345      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754036151.098375      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ Libraries (including Whisper for audio) imported. Dynamo disabled.


In [4]:
#!ls /kaggle/input/

In [5]:
#!ls /kaggle/input/paligemma-2/
#!ls /kaggle/input/paligemma-2/transformers/
#!ls /kaggle/input/paligemma-2/transformers/paligemma2-3b-mix-448/
#!ls /kaggle/input/paligemma-2/transformers/paligemma2-3b-mix-448/1

# Load the Model

In [6]:
# =================================================================
# CELL 2: LOAD THE MODEL (FINAL-FINAL ROBUST VERSION)
# =================================================================

import os

model_path = "/kaggle/input/paligemma-2/transformers/paligemma2-3b-mix-448/1"

model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_path,
    #torch_dtype=torch.bfloat16,
    #device_map="auto",
    local_files_only=True,
).eval()

processor = PaliGemmaProcessor.from_pretrained(
    model_path,
    local_files_only=True
)

print("✅ PaliGemma Model and Processor Loaded")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


✅ PaliGemma Model and Processor Loaded


# The Interactive UI


In [7]:
# =================================================================
# CELL 3: THE COMPLETE INTERACTIVE APPLICATION (WITH PRE-LOADED DEMO)
# =================================================================

# --- Dynamo Fix ---
import torch._dynamo
torch._dynamo.disable()
# ------------------

# Load the whisper model once
whisper_model = whisper.load_model("tiny.en")
print("✅ Whisper Speech-to-Text model loaded.")

# --- UI COMPONENTS ---
# We now add a second button for the demo
prompt_input = widgets.Text(value='Identify the disease on this plant leaf.', description='Prompt:', layout=widgets.Layout(width='80%'))
image_uploader = widgets.FileUpload(accept='image/*', description='Upload Image:')
audio_uploader = widgets.FileUpload(accept='audio/*', description='Upload Audio Note:')
run_custom_button = widgets.Button(description="Run Analysis on Uploaded Files", button_style='success')
run_demo_button = widgets.Button(description="► Load & Run Pre-Loaded Demo", button_style='info') # New demo button
output_area = widgets.Output()


# --- ANALYSIS FUNCTION ---
# This function is now more flexible. It takes image and audio data as arguments.
def run_analysis_pipeline(image_data, audio_data, question_text):
    with output_area:
        output_area.clear_output()

        print("🤖 Transcribing audio notes...")
        # Save audio to a temporary file for whisper to process
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as temp_audio:
            temp_audio.write(audio_data)
            temp_audio.flush()
            transcribed_text = whisper_model.transcribe(temp_audio.name)['text']
        
        print(f"🎤 Farmer's Notes (Transcribed): '{transcribed_text}'")
        print("🤖 Analyzing with PaliGemma...")

        # Convert image data to a usable Image object
        image = Image.open(io.BytesIO(image_data))
        
        # Create the full prompt
        full_prompt = f"<image> {question_text}\nSpoken notes from the farmer: {transcribed_text}"
        
        # Run the model
        inputs = processor(text=full_prompt, images=image, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=128)
        
        # Clean the output
        text_part_of_prompt = f"{question_text}\nSpoken notes from the farmer: {transcribed_text}"
        raw_output_text = processor.decode(outputs[0], skip_special_tokens=True)
        final_answer = raw_output_text[len(text_part_of_prompt):].strip()

        # Display the final, clean result
        output_area.clear_output()
        display(Markdown(f"""
### 🌿 FieldScribe AI Analysis:

**Your Question:**
`{question_text}`

**Farmer's Spoken Notes:**
*`{transcribed_text}`*

**AI Answer:**
**{final_answer}**
"""))


# --- BUTTON HANDLERS ---
# Function for the "Run Analysis on Uploaded Files" button
def handle_custom_run(b):
    if not image_uploader.value or not audio_uploader.value:
        with output_area:
            output_area.clear_output()
            print("❌ Please upload both an image and an audio file to run a custom analysis.")
        return
    # Get data from the uploader widgets
    image_data = image_uploader.value[0]['content']
    audio_data = audio_uploader.value[0]['content']
    question = prompt_input.value
    run_analysis_pipeline(image_data, audio_data, question)

# Function for the "Load & Run Pre-Loaded Demo" button
def handle_demo_run(b):
    with output_area:
        output_area.clear_output()
        print("🚀 Loading pre-loaded demo assets...")
    
    # Define the paths to your demo files
    demo_image_path = "/kaggle/input/fieldscribe-assets/tomato_leaf.jpg"
    demo_audio_path = "/kaggle/input/fieldscribe-assets/farmer_note.wav"
    
    # Read the file data from the paths
    with open(demo_image_path, "rb") as f:
        image_data = f.read()
    with open(demo_audio_path, "rb") as f:
        audio_data = f.read()
        
    question = prompt_input.value
    # Run the main analysis pipeline with the demo data
    run_analysis_pipeline(image_data, audio_data, question)


# --- FINAL SETUP ---
# Connect the buttons to their respective handler functions
run_custom_button.on_click(handle_custom_run)
run_demo_button.on_click(handle_demo_run)

# Display the UI components in a structured way
display(
    Markdown("### Run a Pre-Loaded Demo"),
    run_demo_button,
    Markdown("---"),
    Markdown("### Or, Run with Your Own Files"),
    prompt_input,
    image_uploader,
    audio_uploader,
    run_custom_button,
    output_area
)

100%|█████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 81.5MiB/s]


✅ Whisper Speech-to-Text model loaded.


### Run a Pre-Loaded Demo

Button(button_style='info', description='► Load & Run Pre-Loaded Demo', style=ButtonStyle())

---

...

Text(value='Identify the disease on this plant leaf.', description='Prompt:', layout=Layout(width='80%'))

FileUpload(value=(), accept='image/*', description='Upload Image:')

FileUpload(value=(), accept='audio/*', description='Upload Audio Note:')

Button(button_style='success', description='Run Analysis on Uploaded Files', style=ButtonStyle())

Output()